In [1]:
import pandas as pd
import jieba.analyse
import jieba
# 有的环境配置下read_table出错，因此改用read_csv
raw = pd.read_csv("金庸-射雕英雄传txt精校版.txt",
                  names = ['txt'], sep ='aaa', encoding ="utf-8" ,engine='python')
# 章节判断用变量预处理
def m_head(tmpstr):
    return tmpstr[:1]

def m_mid(tmpstr):
    return tmpstr.find("回 ")

raw['head'] = raw.txt.apply(m_head)
raw['mid'] = raw.txt.apply(m_mid)
raw['len'] = raw.txt.apply(len)
# raw['chap'] = 0
# 章节判断
chapnum = 0
for i in range(len(raw)):
    if raw['head'][i] == "第" and raw['mid'][i] > 0 and raw['len'][i] < 30 :
        chapnum += 1
    if chapnum >= 40 and raw['txt'][i] == "附录一：成吉思汗家族" :
        chapnum = 0
    raw.loc[i, 'chap'] = chapnum
    
# 删除临时变量
del raw['head']
del raw['mid']
del raw['len']
raw.head(50)

words3=list(raw[raw.chap == 3].txt)
words4=list(raw[raw.chap == 4].txt)
words3_k=[]
words4_k=[]
#提取第三章第四章关键词
for w in words3:
    words3_k.append(jieba.analyse.extract_tags(w,20))
for w in words4:
    words4_k.append(jieba.analyse.extract_tags(w,20))



Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 2.038 seconds.
Prefix dict has been built succesfully.


In [2]:
#训练集和测试集的选取
text_words3_k=words3_k[-15:]
train_words3_k=words3_k[:-15]
text_words4_k=words4_k[-15:]
train_words4_k=words4_k[:-15]
#模型的构建
A={}
B={}
C={}
for i in train_words3_k:
    for j in i:
        if j not in A.keys():
            A[j]=0
for i in train_words4_k:
    for j in i:
        if j not in A.keys():
            A[j]=0
for a in A.keys():
    for i in train_words3_k:
        if a in i:
            A[a]=A[a]+1
    for i in train_words4_k:
        if a in i:
            A[a]=A[a]+1
    A[a]=A[a]/(len(train_words3_k)+len(train_words4_k))
B['第三章']=len(train_words3_k)/(len(train_words3_k)+len(train_words4_k))
B['第四章']=1-B['第三章']

In [5]:
for i in A.keys():
    C[i]=0
for key in C.keys():
    for j in train_words3_k:
        if key in j:
            C[key]+=1
    C[key]=C[key]/len(train_words3_k)
C_4={}
for i in A.keys():
    C_4[i]=0
for key in C_4.keys():
    for j in train_words4_k:
        if key in j:
            C_4[key]+=1
    C_4[key]=C_4[key]/len(train_words4_k)
end=[]
endTrue=[]
for i in text_words3_k:
    u=0
    p=0
    for j in i:
        if j not in A.keys():
            continue
        p=p+C[j]*B['第三章']/A[j]
        u=u+1
    if u!=0:
        p=p/u
    else:
        p=0
    if p<0.5:
        end.append('4')
    else:
        end.append("3")
    endTrue.append('3')
for i in text_words4_k:
    u=0
    p=0
    for j in i:
        if j not in A.keys():
            continue
        p=p+C_4[j]*B['第四章']/A[j]
        u=u+1
    if u!=0:
        p=p/u
    else:
        p=0
    if p<0.5:
        end.append('3')
    else:
        end.append("4")
    endTrue.append('4')
# 进行测验
from sklearn.metrics import classification_report
target_names = ['class 3', 'class 4',]
print("第一问\n",classification_report(endTrue, end, target_names=target_names))

第一问
               precision    recall  f1-score   support

     class 3       0.85      0.73      0.79        15
     class 4       0.76      0.87      0.81        15

    accuracy                           0.80        30
   macro avg       0.81      0.80      0.80        30
weighted avg       0.81      0.80      0.80        30



In [18]:
len(end)

30

In [25]:
endTrue

['3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '3',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4']

In [6]:
# 有的环境配置下read_table出错，因此改用read_csv
raw2 = pd.read_csv("附件3_金庸-天龙八部txt精校版.txt",
                  names = ['txt'], sep ='aaa', encoding ="utf-8" ,engine='python')
def A(Astr):
    for i in Astr.split(' ')[0]:
        if i not in '一二三四五六七八九十' or len(Astr.split(' ')[0])>=4:
            return ''
        else:
            return Astr.split(' ')[0]
    
raw2['p']=raw2.txt.apply(A)
raw2_q=[]
qNum=0
for i in list(raw2['p']):
    if i!='':
        qNum+=1
    if qNum>=50:
        qNum=0
    raw2_q.append(qNum)
raw2['chap']=raw2_q
# 3天龙，4射雕
words3=list(raw[raw.chap == 3].txt)
words4=list(raw2[raw2.chap == 3].txt)
words3_k=[]
words4_k=[]
for w in words3:
    words3_k.append(jieba.analyse.extract_tags(w,20))
for w in words4:
    words4_k.append(jieba.analyse.extract_tags(w,20))
text_words3_k=words3_k[-15:]
train_words3_k=words3_k[:-15]
text_words4_k=words4_k[-15:]
train_words4_k=words4_k[:-15]
#模型的构建
A={}
B={}
C={}
for i in train_words3_k:
    for j in i:
        if j not in A.keys():
            A[j]=0
for i in train_words4_k:
    for j in i:
        if j not in A.keys():
            A[j]=0
for a in A.keys():
    for i in train_words3_k:
        if a in i:
            A[a]=A[a]+1
    for i in train_words4_k:
        if a in i:
            A[a]=A[a]+1
    A[a]=A[a]/(len(train_words3_k)+len(train_words4_k))
B['第三章']=len(train_words3_k)/(len(train_words3_k)+len(train_words4_k))
B['第四章']=1-B['第三章']
for i in A.keys():
    C[i]=0
for key in C.keys():
    for j in train_words3_k:
        if key in j:
            C[key]+=1
    C[key]=C[key]/len(train_words3_k)
C_4={}
for i in A.keys():
    C_4[i]=0
for key in C_4.keys():
    for j in train_words4_k:
        if key in j:
            C_4[key]+=1
    C_4[key]=C_4[key]/len(train_words4_k)
end=[]
endTrue=[]
for i in text_words3_k:
    u=0
    p=0
    for j in i:
        if j not in A.keys():
            continue
        p=p+C[j]*B['第三章']/A[j]
        u=u+1
    if u!=0:
        p=p/u
    else:
        p=0
    if p<0.5:
        end.append('4')
    else:
        end.append("3")
    endTrue.append('3')
for i in text_words4_k:
    u=0
    p=0
    for j in i:
        if j not in A.keys():
            continue
        p=p+C_4[j]*B['第四章']/A[j]
        u=u+1
    if u!=0:
        p=p/u
    else:
        p=0
    if p<0.5:
        end.append('3')
    else:
        end.append("4")
    endTrue.append('4')
from sklearn.metrics import classification_report
target_names = ['class 射雕', 'class 天龙',]
print("第一问\n",classification_report(endTrue, end, target_names=target_names))

第一问
               precision    recall  f1-score   support

    class 射雕       0.94      1.00      0.97        15
    class 天龙       1.00      0.93      0.97        15

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



In [ ]:
# 求PYA,P(Y|A,B,C)= P(A|Y)* P(B|Y)* P(C|Y)*P(Y)=概率
# PYA3={}
# for i in PA3.keys():
#     PYA3[i]=0
# for key in PYA3.keys():
#     for j in train_chap3:
#         if key in j:
#             PYA3[key]+=1
#     PYA3[key]=PYA3[key]/len(train_chap3)
# PYA4={}
# for i in PA4.keys():
#     PYA4[i]=0
# for key in PYA4.keys():
#     for j in train_chap4:
#         if key in j:
#             PYA4[key]+=1
#     PYA4[key]=PYA4[key]/len(train_chap4)

In [ ]:
#         u=u+1
#     if u!=0:
#         p=p/u
#     else:
#         p=0
#     if p<0.5:
#         end.append('3')
#     else:
#         end.append("4")
#     endTrue.append('4')